In [ ]:
from models.unet import Unet
from models.diffusion_model import DiffusionModel, DiffusionModelExtended, DiffusionModelTest
#from diffusion_model_improved import TargetDiffusion, LearnedVarDiffusion
import torch
from utils.trainer import Trainer
import numpy as np


In [ ]:
num_classes = len(np.arange(0, 10))

# you can always set num_classes 
unet_model = Unet(dim=32, channels=1, dim_mults=(1, 2, 2), resnet_block_groups=2, num_classes=num_classes, use_sinposemb=False) 
#unet_model = Unet(dim=32, channels=1, dim_mults=(1, 2, 4), resnet_block_groups=8, num_classes=num_classes, use_sinposemb=False, use_learned_var=True)# 8 blocks
# for 
#unet_model = Unet(dim=32, channels=1, dim_mults=(1, 2, 2), resnet_block_groups=2, num_classes=num_classes, use_sinposemb=False, self_condition=True) 
#optimizer = torch.optim.Adam(unet_model.parameters(), lr=2e-5)
optimizer = torch.optim.Adam(unet_model.parameters(), lr=1e-4)

diffusion_model = DiffusionModelExtended(model=unet_model, image_size=32, in_channels=1, timesteps=1000, beta_schedule='linear', loss_weighing=True, min_snr_gamma=5, offset_noise_strength=0.0)
#diffusion_model = DiffusionModel(model=unet_model, image_size=32, in_channels=1, timesteps=1000, beta_schedule='linear')
diffusion_model = DiffusionModelTest(model=unet_model, image_size=32, in_channels=1, timesteps=1000, beta_schedule='linear', use_cfg_me=True)
#diffusion_model = TargetDiffusion(model=unet_model, image_size=32, in_channels=1, timesteps=10, beta_schedule='linear', objective='pred_noise', min_snr_loss_weight=True)
#diffusion_model = LearnedVarDiffusion(model=unet_model, image_size=32, in_channels=1, timesteps=10, beta_schedule='linear', objective='pred_noise', min_snr_loss_weight=True)



In [3]:
trainer = Trainer(diffusion_model=diffusion_model, optimizer=optimizer, use_ema=True, use_guided_diff=True, nb_epochs=2, batch_size=64, device='cpu')
trainer.train_loop()
# model saved with this:
# unet_model = Unet(dim=128, channels=1, dim_mults=(1, 2, 4), resnet_block_groups=8, num_classes=num_classes, use_sinposemb=False) 
# diffusion_model = DiffusionModel(model=unet_model, image_size=32, in_channels=1, timesteps=1000, beta_schedule='linear')
# trainer.load("checkpoints/saved_epoch_1_model.pt")
# trainer.sampling(epoch=1, n_samples=16)

"""
trainer.load("checkpoints/epoch_0_model.pt")
trainer.sampling(epoch=1)
trainer.nb_epochs = 2
trainer.train_loop()
trainer.load("checkpoints/epoch_1_model.pt")
print("trainer.diffusion_model.timesteps", trainer.diffusion_model.timesteps)
"""
# right now: training with dome classifier free guidance implementation => had a mistake in rand_zeros(x)
# right now: training extend without offset-noise
# right now: training with dome classifier free guidance implementation 
# ToDo: right now: training extend without offset-noise => noise reingeben anstatt x_0
# rn: old cfg method => dna-dif
# late nite: dome method mit scaling



Training Loop:   0%|          | 1/938 [00:21<5:29:38, 21.11s/it]

In [ ]:

image = torch.randn(10, 1, 32, 32)
t = (torch.rand(10) * 10).long()
unet = Unet(dim=128, dim_mults=(1, 2, 2), channels=1, resnet_block_groups=8, learned_sinusoidal_dim=18, num_classes=None)
num_param = sum([p.numel() for p in unet.parameters()])
print(num_param)
x_after = unet(image, t)
print(x_after.shape)
